<img src="files/night_sky_eye_small.jpg">
# Image_comparison Script

Jessica Metzger (UChicago) and Jim Annis (Fermilab) 2018
Antonella Palmese and Jim Annis (Fermilab) 2019


To run this notebook:
  * you'll need to run 
       ** conda activate des18a"
  * really, just try sourcing setup.sh
       
 *(to run a notebook remotely, see 
 http://home.fnal.gov/~kadrlica/fnalstart.html
 
 ## How this should work
 
 * Given a search image:
 * Select from the 200mpc catalog the likely galaxies in the image as a list of ra,dec's
 * Select the galaxy from the bias subtracted search image, compare to BLISS single epoch images



# Do the Eyeballing!

This is the action cell!

For a given image, get the 
image name, ra, dec

Change the triplets,
then run the cell, it will take time to do all of them

In [5]:
import sauron

# you do know the search image ra,dec, right? and filter? and name?
image_ra, image_dec =91.841025,-35.34666
image_name = "856770"; filter="r"

# get your 200mpc++ catalog into place
gal_list = "/data/des60.b/data/palmese/GW_host_catalog/S190814bv/200Mpc_BPortal_2sigma_ranked.csv"
# tell me what columns to use
# ID dpDV RA DEC DIST Z ZERR CATTYPE
id_col = 0
ra_col = 2
dec_col = 3

# eyeball
work_image(image_name, image_ra,image_dec, gal_list=gal_list,
                id_col_num=id_col, ra_col_num=ra_col, dec_col_num=dec_col,
                filter=filter, do_num=-1)